# Session 4.2 v2: Interacting with a Deployed BakeryAI API

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1VviWwCNNrFHAFCkkLOnLpg60Y0orste7?usp=sharing)

## 🎯 Goal: From Notebook to Production API Client

This notebook will serve as a client to interact with our deployed `BakeryAI` LangServe application. We will cover:

1.  **Local Docker Interaction**: How to call the API running in a local Docker container.
2.  **Cloud Deployment Interaction**: How to modify the client to call the API once it's deployed to the cloud.



## 1. Setup and Installation

First, let's install the necessary libraries to interact with our LangServe API.



In [1]:
%pip install -q "langserve[all]" langchain_openai python-dotenv


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.1/467.1 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.1 which is incompatible.


## 2. Running the BakeryAI API with Docker

For context, ther repo is https://github.com/IvanReznikov/mdx-langchain-conclave

Before we can interact with the API, we need to run it. Open a terminal, navigate to the `deployment` directory, and run the following command:

```bash
docker-compose up --build
```

This will build the Docker image and start the BakeryAI API service. The API will be available at `http://localhost:8000`.



## 3. Interacting with the Local API

Now that the API is running, we can interact with it using the `langserve.RemoteRunnable`. This allows us to treat the remote LangChain object as if it were running locally in our notebook.



In [2]:
from langserve import RemoteRunnable

LOCALHOST = False
if LOCALHOST:
  # Connect to the local chat endpoint
  remote_chain = RemoteRunnable("http://localhost:8000/chat/")

  # Invoke the chain
  result = remote_chain.invoke({"question": "Do you have any gluten free options?"})
  print("Response:", result)




In [3]:
if LOCALHOST:
  # Stream responses
  print("Streaming response:")
  for chunk in remote_chain.stream({"question": "Give me a recomendation for a birthday party for my son. He is turning 10. He likes to play soccer and he is a big fan of Spiderman. We expect 7 guests."}):
      print(chunk, end="", flush=True)



## 4. Interacting with a Cloud-Deployed API

Once you have deployed the `BakeryAI` application to a cloud provider (like AWS, GCP, Azure, etc.), you will get a public URL for your API. To interact with the deployed API, you simply need to replace the local URL (`http://localhost:8000`) with the public URL.

For example, if your API is deployed at `https://your-bakery-api.com`, you would connect to it like this:



In [4]:
import os

# It's a good practice to store the API URL in an environment variable
# You would set this in your environment, or a .env file
# For example: BAKERY_API_URL="https://your-bakery-api.com"
# For this example, we'll just use a placeholder.
# In a real scenario, you would load this from the environment.
# from dotenv import load_dotenv
# load_dotenv()
# cloud_api_url = os.getenv("BAKERY_API_URL")

cloud_api_url = "https://bakery-api-696156688857.us-central1.run.app" # <--- IMPORTANT: Replace this with your actual URL

# you can check if the url is set
if "replace-with-your-deployed-api-url.com" in cloud_api_url:
    print("Please replace the placeholder URL with your deployed API URL.")

In [5]:
# Invoke the chain
try:
    cloud_chain = RemoteRunnable(f"{cloud_api_url}/chat/")
    result = cloud_chain.invoke({"question": "What is the most popular cupcake flavor?"})
    print("Response from cloud API:")
    print(result)
except Exception as e:
    print(f"An error occurred: {e}")
    print("Please ensure your API is deployed and the URL is correct.")

Response from cloud API:
The most popular cupcake flavor tends to vary by location and personal preference, but some consistently popular flavors include:

1. **Chocolate**: A classic favorite, rich and satisfying.
2. **Vanilla**: Loved for its simplicity and versatility.
3. **Red Velvet**: Known for its subtle cocoa flavor and striking color.
4. **Lemon**: Refreshing and tangy, often favored in warmer months.
5. **Salted Caramel**: Combines sweet and salty for a decadent treat.

These flavors are commonly sought after in bakeries and are considered crowd-pleasers.


In [6]:
# Connect to the RAG endpoint
try:
    rag_chain = RemoteRunnable(f"{cloud_api_url}/rag/")
    result = rag_chain.invoke({"question": "does the blast berry bomb cake contain allergens?"})
    print("\n\nResponse from RAG endpoint:")
    print(result)
except Exception as e:
    print(f"\n\nError with RAG endpoint: {e}")




Response from RAG endpoint:
Yes, the Birthday Blast Berry Bomb cake contains allergens. It includes nuts, milk, wheat, and eggs.


In [7]:
# Connect to the recommend endpoint
try:
    recommend_chain = RemoteRunnable(f"{cloud_api_url}/recommend/")
    result = recommend_chain.invoke({"occasion": "birthday", "preferences": "chocolate"})
    print("\nResponse from recommend endpoint:")
    print(result)
except Exception as e:
    print(f"\nError with recommend endpoint: {e}")


Response from recommend endpoint:
For a chocolate lover's birthday celebration, I recommend a classic Chocolate Ganache Cake. This decadent cake is perfect for the occasion for several reasons:

1. **Rich Flavor**: A Chocolate Ganache Cake is deeply rich and indulgent, made with high-quality cocoa and topped with a luscious layer of chocolate ganache. This will satisfy any chocolate enthusiast's craving for a pure, intense chocolate experience.

2. **Moist Texture**: The ganache adds a silky, smooth finish to the cake, complementing the moistness of the chocolate sponge layers. This combination makes for a melt-in-your-mouth experience that will delight guests with its texture.

3. **Elegant Presentation**: The glossy surface of the ganache creates a sophisticated and elegant look, ideal for a special birthday celebration. It's simple yet stunning, especially if you choose to decorate it with chocolate shavings, berries, or gold leaf for an added touch of luxury.

4. **Versatility**: 